## Transforms

In [2]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.nn.functional import one_hot

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: one_hot(torch.tensor(y), num_classes=10).float())
)

In [3]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds_orig = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [4]:
ds == ds_orig  # This should be True, confirming the two datasets are equivalent

False

In [9]:
ix = 11

print(ds[ix][1])
print(ds_orig[ix][1])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])


In [11]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}, probability: {pred_probab}, logits: {logits}")

Predicted class: tensor([2]), probability: tensor([[0.1057, 0.0970, 0.1071, 0.0984, 0.0960, 0.1026, 0.1064, 0.0932, 0.0996,
         0.0940]], grad_fn=<SoftmaxBackward0>), logits: tensor([[ 0.0714, -0.0146,  0.0845,  0.0004, -0.0242,  0.0420,  0.0778, -0.0545,
          0.0123, -0.0456]], grad_fn=<AddmmBackward0>)


In [26]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [27]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [28]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [29]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.7281, -0.2490,  0.1300,  0.5590, -0.2850,  0.2521,  0.2552,  0.2139,
          0.2153, -0.0486,  0.0486, -0.1703, -0.3330,  0.4311, -0.1118,  0.1788,
         -0.0095,  0.2345,  0.1659,  0.6303],
        [ 0.6291, -0.1844,  0.1351,  0.2641, -0.1747,  0.1432,  0.1926,  0.4324,
          0.3083, -0.0364,  0.2812,  0.0629, -0.3011,  0.3722,  0.0276, -0.0919,
          0.1273,  0.0232, -0.1784,  0.3443],
        [ 0.5347, -0.1096,  0.2008,  0.2492, -0.1515,  0.3891,  0.1441,  0.1319,
          0.2103, -0.0322, -0.0370,  0.1855, -0.2779,  0.4089,  0.1223,  0.0317,
          0.2963,  0.1675, -0.1725,  0.1425]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.7281, 0.0000, 0.1300, 0.5590, 0.0000, 0.2521, 0.2552, 0.2139, 0.2153,
         0.0000, 0.0486, 0.0000, 0.0000, 0.4311, 0.0000, 0.1788, 0.0000, 0.2345,
         0.1659, 0.6303],
        [0.6291, 0.0000, 0.1351, 0.2641, 0.0000, 0.1432, 0.1926, 0.4324, 0.3083,
         0.0000, 0.2812, 0.0629, 0.0000, 0.3722, 0.02

In [30]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [39]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)


In [40]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0181, -0.0146,  0.0090,  ..., -0.0143,  0.0045, -0.0309],
        [-0.0049, -0.0277, -0.0117,  ..., -0.0127,  0.0042,  0.0250]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0285, -0.0038], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0420, -0.0221,  0.0160,  ...,  0.0048, -0.0073,  0.0386],
        [-0.0091,  0.0019,  0.0208,  ..., -0.0214,  0.0154,  0.0282]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 